In [2]:
from typing import Any, Type

from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.tools import DuckDuckGoSearchResults

# 3.5는 결과가 틀림
llm = ChatOpenAI(temperature=0.1)


class StockMarketSymbolSearchToolArgsSchema(BaseModel):
    query: str = Field(
        description="The query you will search for.Example query: Stock Market Symbol for Apple Company"
    )


class StockMarketSymbolSearchTool(BaseTool):
    name = "StockMarketSymbolSearchTool"
    description = """
    Use this tool to find the stock market symbol for a company.
    It takes a query as an argument.
    
    """
    args_schema: Type[
        StockMarketSymbolSearchToolArgsSchema
    ] = StockMarketSymbolSearchToolArgsSchema

    def _run(self, query):
        # Web Search API
        # brew install ddgr
        ddg = DuckDuckGoSearchResults()
        return ddg.run(query)


"""
agent에 input을 줄 때 runtime은 LLM으로 부터 get_action을 호출해서 LLM이 선택한 Tool을 돌림
AgentFinish가 될떄까지 loop를 돌리기 떄문에 Cost가 많이 나올 수 있음.
가끔 LLM이 Json이 아닌 Text로 답할 때가 있음. 그러땐 Agent가 에러가 남
"""
# llm, tool, agent를 전달해야 함.
agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS, # OpenAI Agent
    handle_parsing_errors=True, # LLM의 output 때문에 Parsing Error가 발생하면 이를 수정하려고 함.
    tools=[
        StockMarketSymbolSearchTool(),
    ],
)


prompt = "Give me information on Cloudflare's stock and help me analyze if it's a potential good investment. Also tell me what symbol does the stock have."

agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `StockMarketSymbolSearchTool` with `{'query': 'Cloudflare stock'}`


[snippet: Stock analysis for Cloudflare Inc (NET:New York) including stock price, stock chart, company news, key statistics, fundamentals and company profile., title: NET: Cloudflare Inc Stock Price Quote - New York - Bloomberg, link: https://www.bloomberg.com/quote/NET:US], [snippet: Cloudflare (NYSE:NET) Founded by two grad students of Harvard Business School, Cloudflare (NYSE:NET) is a software as a service platform that helps improve security, reliability and loading times ..., title: Winners And Losers Of Q4: Cloudflare (NYSE:NET) Vs The Rest Of The ..., link: https://finance.yahoo.com/news/winners-losers-q4-cloudflare-nyse-084627560.html], [snippet: At Cloudflare, Inc., we promise to treat your data with respect and will not share your information with any third party. You can unsubscribe to any of the investor alerts you are subscribed to by visiting the 'unsub

{'input': "Give me information on Cloudflare's stock and help me analyze if it's a potential good investment. Also tell me what symbol does the stock have.",
 'output': "Cloudflare's stock symbol is NET and it is listed on the New York Stock Exchange (NYSE). \n\nTo analyze if Cloudflare is a potential good investment, we can look at various factors such as the company's financial performance, growth prospects, competitive position, and market trends. \n\nHere are some resources where you can find more information on Cloudflare's stock:\n1. [Bloomberg - Cloudflare Inc Stock Price Quote](https://www.bloomberg.com/quote/NET:US)\n2. [Yahoo Finance - Cloudflare (NYSE:NET)](https://finance.yahoo.com/news/winners-losers-q4-cloudflare-nyse-084627560.html)\n3. [Cloudflare, Inc. - Stock Quote & Chart](https://cloudflare.net/stock-information/Stock-Quote--Chart/default.aspx)\n4. [Stock Analysis - Cloudflare, Inc. (NET) Stock Price, Quote & News](https://stockanalysis.com/stocks/net/)\n\nI recomme